In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
filmes = pd.read_csv('movies_metadata.csv', low_memory = False)
def generos(df):
    if isinstance(df, str):
        df=ast.literal_eval(df)
        nomes=[item['name']for item in df]
        return nomes
    return []
def pegar_nome(df):
    try:
        companies=ast.literal_eval(df)
        nomes=[company['nomes']for company in companies]
        return nomes
    except(ValueError, TypeError):
        return[]
filmes['genres'] = filmes['genres'].apply(generos)
filmes.rename(columns = {'id':'ID_FILME'}, inplace = True)
filmes = filmes[filmes['vote_count'] > 999]
filmes= filmes[['ID_FILME', 'title', 'vote_count']]
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)
filmes.head()

,ID_FILME,title,vote_count
0,862,Toy Story,5415.0
1,8844,Jumanji,2413.0
5,949,Heat,1886.0
9,710,GoldenEye,1194.0
15,524,Casino,1343.0


In [4]:
avaliacoes= pd.read_csv('ratings.csv', low_memory= False)
avaliacoes= avaliacoes[['userId','movieId','rating']]
avaliacoes.rename(columns = {'movieId':'ID_FILME'}, inplace = True)
avaliacoes

,userId,ID_FILME,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [5]:
qt_avaliacoes = avaliacoes['userId'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index
avaliacoes = avaliacoes[avaliacoes['userId'].isin(y)]
avaliacoes.drop_duplicates(['userId','ID_FILME'], inplace = True)
avaliacoes

C:\Users\Julio\AppData\Local\Temp\ipykernel_10172\3017818884.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avaliacoes.drop_duplicates(['userId','ID_FILME'], inplace = True)


,userId,ID_FILME,rating
17291,229,1,3.0
17292,229,2,3.0
17293,229,4,2.0
17294,229,5,1.0
17295,229,7,2.0
...,...,...,...
26023517,270887,171439,1.0
26023518,270887,171755,3.0
26023519,270887,173149,5.0
26023520,270887,173405,5.0


In [6]:
ava_filmes= avaliacoes.merge(filmes, on= 'ID_FILME')

ava_filmes

,userId,ID_FILME,rating,title,vote_count
0,229,12,1.0,Finding Nemo,6292.0
1,741,12,3.0,Finding Nemo,6292.0
2,1932,12,0.5,Finding Nemo,6292.0
3,3437,12,2.0,Finding Nemo,6292.0
4,3694,12,1.5,Finding Nemo,6292.0
...,...,...,...,...,...
193039,243443,8909,1.0,Wanted,2583.0
193040,218368,101299,4.0,The Hunger Games: Catching Fire,6656.0
193041,219677,146216,1.0,RED 2,1548.0
193042,221327,118340,4.0,Guardians of the Galaxy,10014.0


In [7]:
ava_filmes.describe()

,userId,ID_FILME,rating,vote_count
count,193044.000000,193044.000000,193044.000000,193044.000000
mean,133152.136166,2423.056650,3.207178,2585.826812
std,77788.050219,7674.938613,1.077808,1630.668466
min,229.000000,11.000000,0.500000,1005.000000
25%,65876.000000,329.000000,2.500000,1438.000000
50%,132001.000000,674.000000,3.000000,2000.000000
75%,200222.000000,1949.000000,4.000000,3223.000000
max,270887.000000,173897.000000,5.000000,14075.000000


In [8]:
pivot_filmes = ava_filmes.pivot_table(columns = 'title', index ='userId', values = 'rating')
pivot_filmes

title,10 Things I Hate About You,12 Angry Men,127 Hours,1408,2 Fast 2 Furious,2001: A Space Odyssey,27 Dresses,28 Days Later,28 Weeks Later,300,...,Wanted,War of the Worlds,Warrior,What Women Want,Who Framed Roger Rabbit,Wild Wild West,X-Men Origins: Wolverine,X-Men: Days of Future Past,Zodiac,xXx
userId,,,,,,,,,,,,,,,,,,,,,
229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN
231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0,2.0,NaN,...,NaN,NaN,NaN,2.5,NaN,NaN,4.0,NaN,NaN,4.0
836,NaN,NaN,NaN,NaN,NaN,2.5,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,2.5,NaN,NaN,3.5,NaN,NaN,3.5
1104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270213,NaN,NaN,NaN,2.0,NaN,NaN,NaN,3.0,3.0,4.0,...,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,NaN
270237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
270564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pivot_filmes.fillna(0, inplace=True)
pivot_filmes

title,10 Things I Hate About You,12 Angry Men,127 Hours,1408,2 Fast 2 Furious,2001: A Space Odyssey,27 Dresses,28 Days Later,28 Weeks Later,300,...,Wanted,War of the Worlds,Warrior,What Women Want,Who Framed Roger Rabbit,Wild Wild West,X-Men Origins: Wolverine,X-Men: Days of Future Past,Zodiac,xXx
userId,,,,,,,,,,,,,,,,,,,,,
229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0
231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0
741,0.0,0.0,0.0,0.0,0.0,3.5,0.0,2.0,2.0,0.0,...,0.0,0.0,0.0,2.5,0.0,0.0,4.0,0.0,0.0,4.0
836,0.0,0.0,0.0,0.0,0.0,2.5,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,2.5,0.0,0.0,3.5,0.0,0.0,3.5
1104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270213,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,3.0,4.0,...,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0
270237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
270564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [11]:
#essa função vai retornar os usuários mais similares de acordo com o usuário selecionado
def users_similares(id_usuario,matrix,k=5):
    #criar um df somente para o user que vou recomendar
    usuario = matrix[matrix.index == id_usuario]
    #criar um df para o resto
    outros= matrix[matrix.index != id_usuario]
    #calculo da similaridade 
    similar= cosine_similarity(usuario, outros)[0].tolist()
    #fazendo um indice para cada usuário
    indice= outros.index.tolist()
    #fazer um dicionário com as  pares de user e os indices de similariade
    similaridade_index= dict(zip(indice,similar))
    #ordenado por similaridade
    usuarios_mais_similares = sorted(similaridade_index.items(), key=operator.itemgetter(1))
    usuarios_mais_similares.reverse()
    
    # colocar usuários mais similares no topo
    topo = usuarios_mais_similares[:k]
    usuarios=[u[0] for u in topo]
    return usuarios
usuario_atual= 229
similares = users_similares(usuario_atual,pivot_filmes)
#print(similares)
type(similares)

list

## Recomendando Filmes

In [12]:
def recomendacoes(index_usuario, usuario_similar, matrix, items=10):
    #carregar os vetores dos users similares
    usuario_similar= matrix[matrix.index.isin(similares)]
    #pegar a media do intens usando a users similares
    usuario_similar= usuario_similar.mean(axis=0)
    #transforamndo em um df
    usuario_similar_df=pd.DataFrame(usuario_similar, columns=['média'])
    #colocand o vetor para o usuário que vamos recomendar
    usuario_df=matrix[matrix.index == index_usuario]
    #facilitar a filtragem
    usuario_df_t= usuario_df.transpose()
    #renomear a coluna
    usuario_df_t.columns=['Avaliações']
    usuario_df_t= usuario_df_t[usuario_df_t['Avaliações'] == 0]
    #lsita de filmes não avaliados
    filmes_nao_vistos= usuario_df_t.index.tolist()
    
    usuarios_simi_df_f= usuario_similar_df[usuario_similar_df.index.isin(filmes_nao_vistos)]
    usuarios_simi_ord= usuarios_simi_df_f.sort_values(by=['média'], ascending=False)
    melhores_filmes= usuarios_simi_ord.head(items)
    return melhores_filmes
recomendacao= recomendacoes(229, similares,pivot_filmes)

In [13]:
display(recomendacao)

,média
title,
Scarface,4.0
The Bourne Supremacy,3.8
Finding Neverland,3.8
Back to the Future Part II,3.2
The Butterfly Effect,3.1
Ocean's Twelve,3.0
Interview with the Vampire,2.8
Sunshine,2.7
Edward Scissorhands,2.7
